Hadoop/MapReduce -- WordCount en Java
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/big-data-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/big-data-analytics/tree/master/) para explorar el repositorio usando `nbviewer`.

# Definición del problema

Se desea contar la frecuencia de ocurrencia de palabras en un conjunto de documentos. Debido a los requerimientos de diseño (gran volúmen de datos y tiempos rápidos de respuesta) se desea implementar una arquitectura Big Data.

A continuación se generarán tres archivos de prueba para probar el sistema.

In [1]:
## Se crea el directorio de entrada
!rm -rf input
!rm -rf output
!mkdir input

In [2]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [3]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [4]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing input/text2.txt


# Solución

Un tutorial detallado se encuentra en http://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html

En este tutorial se planteará la solución en Java. Para construir una aplicación que usa MapReduce, el usuario debe implementar la función map y la función reduce; el sistema se encarga por si solo de ejecutarlas en el cluster. La implementación es la siguiente:

In [5]:
%%writefile WordCount.java

import java.io.IOException;

/*
 * Esta clase permite separar una frase (texto)
 * en las palabras que lo conforman. La lista
 * resultante puede ser iterada en un ciclo for
 */
import java.util.StringTokenizer;

/*
 *
 * Librerias requeridas para ejecutar Hadoop
 *
 */
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

/*
 * Esta clase implementa el mapper y el reducer
 */
public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{
       
    private final static IntWritable one = new IntWritable(1);

    /* 
     * en esta variable se guarda cada palabra leida        
     * del flujo de entrada
     */     
    private Text word = new Text();

    /* 
     * Este es el mapper. Para cada palabra 
     * leída, emite el par <word, 1>
     */
    public void map(Object key,       // Clave
                    Text value,       // La linea de texto
                    Context context   // Aplicación que se esta ejecutando
                    ) throws IOException, InterruptedException {
                              
      // Convierte la línea de texto en una lista de strings
      StringTokenizer itr = new StringTokenizer(value.toString());
                              
      // Ejecuta el ciclo para cada palabra 
      // de la lista de strings
      while (itr.hasMoreTokens()) {
        // obtiene la palabra
        word.set(itr.nextToken());

        // escribe la pareja <word, 1> 
        // al flujo de salida
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
           
    // Clase para imprimir un entero al flujo de salida       
    private IntWritable result = new IntWritable();

    // Esta función es llamada para reducir 
    // una lista de valores que tienen la misma clave
    public void reduce(Text key,                      // clave
                       Iterable<IntWritable> values,  // lista de valores
                       Context context                // Aplicación que se esta ejecutando
                       ) throws IOException, InterruptedException {
        
      // itera sobre la lista de valores, sumandolos
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
        
      // escribe la pareja <word, valor> al flujo
      // de salida
      context.write(key, result);
    }
  }

    
  /*
   * Se crea la aplicación en Hadoop y se ejecuta
   */
  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    
    /*
     * El job corresponde a la aplicacion
     */
    Job job = Job.getInstance(conf, "word count");
      
    /*
     * La clase que contiene el mapper y el reducer
     */
    job.setJarByClass(WordCount.class);
      
    /* 
     * Clase que implementa el mapper  
     */ 
    job.setMapperClass(TokenizerMapper.class);
      
    /*
     * El combiner es un reducer que se coloca a la salida
     * del mapper para agilizar el computo
     */
    job.setCombinerClass(IntSumReducer.class);
    
    /*
     * Clase que implementa el reducer
     */
    job.setReducerClass(IntSumReducer.class);
      
    /*
     * Salida
     */
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    
    /*
     * Formatos de entrada y salida
     */
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
   
    // resultado de la ejecución.
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}



Writing WordCount.java


In [6]:
%%writefile WordCount.java

import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{

    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key, Text value, Context context
                    ) throws IOException, InterruptedException {
      StringTokenizer itr = new StringTokenizer(value.toString());
      while (itr.hasMoreTokens()) {
        word.set(itr.nextToken());
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
    private IntWritable result = new IntWritable();

    public void reduce(Text key, Iterable<IntWritable> values,
                       Context context
                       ) throws IOException, InterruptedException {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
      context.write(key, result);
    }
  }

  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    Job job = Job.getInstance(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setReducerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}

Overwriting WordCount.java


In [7]:
## compila el programa
!hadoop com.sun.tools.javac.Main WordCount.java

## genera el archivo *.jar para ejecutarlo en hadoop
!jar cf wc.jar WordCount*.class

## ejecuta el proceso para los archivos 
## de texto en el directorio input
!hadoop jar wc.jar WordCount input output

2018-08-27 16:25:12,169 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-08-27 16:25:12,567 INFO impl.MetricsConfig: loaded properties from hadoop-metrics2.properties
2018-08-27 16:25:12,776 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2018-08-27 16:25:12,776 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2018-08-27 16:25:12,965 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2018-08-27 16:25:13,062 INFO input.FileInputFormat: Total input files to process : 3
2018-08-27 16:25:13,116 INFO mapreduce.JobSubmitter: number of splits:3
2018-08-27 16:25:13,264 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local560905394_0001
2018-08-27 16:25:13,266 INFO mapreduce.JobSubmitter: Executing with tokens: []
2018-08-27 16:25:

2018-08-27 16:25:13,766 INFO mapred.MapTask: Finished spill 0
2018-08-27 16:25:13,773 INFO mapred.Task: Task:attempt_local560905394_0001_m_000002_0 is done. And is in the process of committing
2018-08-27 16:25:13,774 INFO mapred.LocalJobRunner: map
2018-08-27 16:25:13,774 INFO mapred.Task: Task 'attempt_local560905394_0001_m_000002_0' done.
2018-08-27 16:25:13,774 INFO mapred.Task: Final Counters for attempt_local560905394_0001_m_000002_0: Counters: 18
	File System Counters
		FILE: Number of bytes read=6246
		FILE: Number of bytes written=501463
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
	Map-Reduce Framework
		Map input records=4
		Map output records=57
		Map output bytes=577
		Map output materialized bytes=566
		Input split bytes=129
		Combine input records=57
		Combine output records=43
		Spilled Records=43
		Failed Shuffles=0
		Merged Map outputs=0
		GC time elapsed (ms)=39
		Total committed heap usage (bytes)=

In [8]:
## contenido el directorio de salida
!ls output/

_SUCCESS     part-r-00000


In [9]:
## imprime el resultado en pantalla
!cat output/part-r-00000 | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1
The	2
a	1
about	1
aid	1
algorithms	1
analysis,	1
analysis.	1
analytics	8
analytics,	8
analytics.	1
analyze	1
and	15
application	1
apply	1
are	1
areas	2
assortment	1
be	1
big	1
business	4
by	2
call	1
can	2
certain	1
changes.	1
cognitive	1
commercial	1
communication	1
computation	1
computer	2
conclusions	1
contain,	1
credit	1
current	1
data	4
data),	1
data.	1
decision	1
decisions	2
describe,	1
descriptive	1
discovery,	1
disprove	1
draw	1
effects	1
enable	1
enterprise	1
evaluate	1
events,	1
examining	1
extensive	1
field	1
for	1
force	1
fraud	1
gaining	1
given	1
goal	1
harness	1
historical	1
hypotheses.	1
improve	2
improvements	1
in	5
include	1
increasingly	1
industries	1
information	1
information,	1
interpretation,	1
involves	1
is	3
knowledge	1
make	2
management,	1
marketing	2
mathematics.	1
may	1
meaningful	1
methods	1
mix	1
modeling,	2
models,	1
more-informed	1
most	1
of	8
often	1
on

---
**Ejercicio.--** Cómo podría mejorar el código anterior? realice una implementación.

---

In [11]:
## se limpia el directoroio de trabajo
!rm WordCount*.* *.jar
!rm -rf input output

rm: WordCount*.*: No such file or directory
rm: *.jar: No such file or directory


---

Hadoop/MapReduce -- WordCount en Java
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/big-data-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/big-data-analytics/tree/master/) para explorar el repositorio usando `nbviewer`.